# АНАЛИЗ ПРИЧИН УХОДА КЛИЕНТОВ ОТ БАНКА

Для анализа причин ухода клиентов необходимо ответить на вопросы. Для начала прочитаем данные и уберём лишние столбцы 

In [283]:
import pandas as pd

churn_data = pd.read_csv('data/churn.csv')
churn_data = churn_data.drop('RowNumber', axis=1) #убираем столбец RowNumber - в задании он не нужен

import plotly
import plotly.express as px

churn_data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Вопрос №1. 
Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [284]:
pie_data = churn_data.groupby(by='Exited')[['Exited']].count() # считаем количество клиентов по признаку лояльности (1 - лояльные, 0 - ушедшие)

type_of_clients = {0:'Лояльные клиенты', 1:'Ушедшие клиенты'} # вводим словарь для легенды графика

#строим круговую диаграмму, чтобы показать структуру

fig_pie = px.pie(
    data_frame=pie_data, 
    names=type_of_clients,
    values='Exited',
    title='Структура клиентов банка (ушедшие/лояльные)',
    height=500, #высота
    width=500, #ширина
)
fig_pie.show()

## Вывод по Вопросу №1. 
Подавляющее большинство клиентов банка (почти 80%) - это лояльные клиенты. Соотношение количества ушедших и лояльных клиентов близко к 1:4 (20,4% к 79,6%)

## Вопрос №2. 
Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [285]:
mask1 = churn_data['Balance'] > 2500 #делаем маску по размеру баланса

hist_data = churn_data[mask1]['Balance'] #делаем новый df с учётом маски

fig_hist = px.histogram(
    data_frame=hist_data,
    x='Balance',
    nbins=20,
    histnorm='percent', #при необходимости можно добавить комментирование на строку для получения на оси количества клиентов вместо доли в %
    marginal = 'box',
    title='Распределение баланса пользователей, у которых на счету больше $2500'
)

fig_hist.update_xaxes(title_text='Баланс на всех счетах клиента', col=1, row=1) # добавляем подписи на оси 
fig_hist.update_yaxes(title_text='% от количества клиентов', col=1, row=1)


fig_hist.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_hist.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)


fig_hist.show()

## Вывод по Вопросу №2. 
Среди клиентов банка со балансом на счёте больше $2500 медианное значение баланса на счёте клиента составляет $120к. Почти у половины клиентов баланс на счете находится в пределах от $100к до $140к (межквартильный интервал). В соответствии с границами распределения минимальное значение баланса составляет $41к, а максимальное - $198к. Экстремальные значения также встречаются: например, минимальное - $3,7к и $251к, но на них приходится менее 1% клиентов. 

Учитывая сказанное выше можно сказать, что банк ориентирован на предложение банковских продуктов под определённый баланс клиентов (80% клиентов с балансом счёта в диапазоне $80к - $160к).

## Вопрос №3. 
Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [286]:
box_data = churn_data.loc[::, ['Exited', 'Balance']] #создаем df для коробочной диаграммы, оставляя только нужные столбцы

box_data['Exited'] = box_data['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента


fig_box = px.box(
    data_frame=box_data,
    x='Balance',
    color='Exited',
    title='Распределение баланса пользователей по признаку оттока',
    labels={'Exited': 'Признак клиента'},
)

fig_box.update_xaxes(title_text='Баланс на всех счетах клиента', col=1, row=1) # добавляем подписи на оси 

fig_box.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_box.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig_box.show()

## Вывод по Вопросу №3.
Среди ушедших клиентов отмечается более высокое медианное значение баланса на счете ($110к проти $92к), а также меньший разрброс значений в балансе на счёте. Исходя из этого можно сделать вывод, что эти клиенты - более состоятельные и нашли лучшие условия по размещению своих средств, пакету услуг и/или качеству обслуживания в другом банке.

## Вопрос №4. 
Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [287]:
box_data_age = churn_data.loc[::, ['Exited', 'Age']] #создаем df для коробочной диаграммы, оставляя только нужные столбцы

box_data_age['Exited'] = box_data_age['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента

fig_box_age = px.box(
    data_frame=box_data_age,
    x='Age',
    color='Exited',
    title='Распределение возраста пользователей по признаку оттока',
    labels={'Exited': 'Признак клиента'},
)

fig_box_age.update_xaxes(title_text='Возраст клиента', col=1, row=1) # добавляем подписи на оси 

fig_box_age.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_box_age.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig_box_age.show()

## Вывод по Вопросу №4. 
Наибольшее количество потенциальных выбросов зафиксировано в группе лояльных клиентов. Границы распределения возраста в группе лояльных клиентов - 18 лет и 56 лет, а 50% клиентов в возрасте от 31 до 41 года, медианный возраст - 36 лет. В группе ушедших клиентов верхняя граница распределения возраста выше - 71 год. Также выше и медианный возраст (45 лет).\
С учётом этого стоит обратить внимание на работу с более "возрастной" аудиторией - от 56 до 70 лет (по ним идет отличие в выбросах), а также с аудиторией от 40 до 51 года (эта группа клиентов составляет значительную долю в ушедших клиентах, но при этом малую долю - в лояльных клиентах). Обе указанные группы являются клиентами со значительной финансовой подушкой с учетом периода жизни (зрелый и предпенсионный возраты) и потенциальной более доходной группой, что подтверждается графиком к Вопросу №3.

## Вопрос №5. 
Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [288]:
scatter_data = churn_data.loc[::, ['CreditScore', 'EstimatedSalary', 'Exited']] #создаем df для диаграммы распределения

scatter_data['Exited'] = scatter_data['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента

#для иллюстрации зависимости добавил график тренда. чтобы не вызывало ошибку потребуется установка библиотеки statsmodels

scatter_crating_salary = px.scatter(
    data_frame=scatter_data, 
    y = 'CreditScore',
    x = 'EstimatedSalary',
    color='Exited',
    title='Взаимоcвязь предполагаемой зарплаты и кредитного рейтинга',
    labels={'Exited': 'Признак клиента','CreditScore':'Кредитный рейтинг','EstimatedSalary':'Предполагаемая заработная плата'},
    log_x=True, 
    log_y=True,
    trendline='ols', 
    trendline_scope='trace',
    trendline_color_override='green',
)
scatter_crating_salary.show()

## Вывод по Вопросу №5. 
На основе графика можно сделать вывод, что большинство клиентов имеют высокий кредитный рейтинг и и высокую заработную плату (т.к. скопление точек увеличивается в правом верхнем углу). Это отмечается и группе ушедших клиентов и в группе лояльных клиентов. При этом при анализе тренда, иллюстрирующего зависимость от кредитного рейтинга от предполагаемой заработной платы (выделен зелёным) отмечается слабое влияние фактора заработной платы на кредитный рейтинг (график почти не имеет наклона). Указанная зависимость проявляется и в группе ушедших клиентов и в группе лояльных клиентов. Вероятно на кредитный рейтинг оказывают сильное влияние другие факторы.

## Вопрос №6. 
Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

In [289]:
pie_data_gender = churn_data[churn_data['Exited']==1].groupby(by='Gender')['Gender'].count() # считаем количество ушедших клиентов по полу

type_of_clients_gender = {'Female':'Женщины', 'Male':'Мужчины'} # вводим словарь для легенды графика


#строим круговую диаграмму, чтобы показать структуру

fig_pie_gender = px.pie(
    data_frame=pie_data_gender, 
    names=type_of_clients_gender,
    values='Gender',
    title='Структура ушедших клиентов банка по полу',
    height=500, 
    width=500, 
)
fig_pie_gender.show()

## Вывод по Вопросу №6. 
Среди ушедших клиентов (2037 чел.) наибольшую долю занимают женщины - почти 56% (1139 чел.)

## Вопрос №7. 
Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [290]:
hist_data_num_of_products = churn_data.loc[::, ['NumOfProducts', 'Exited']] #делаем новый df 

hist_data_num_of_products = hist_data_num_of_products.sort_values(by='NumOfProducts', ascending=True) # сортируем в порядке возрастания для корретного отображения в дальнейшем на оси абсцисс
hist_data_num_of_products['NumOfProducts']  = hist_data_num_of_products['NumOfProducts'].astype(str) # преобразуем количество в строковый тип для корректного отображения на оси абсцисс

hist_data_num_of_products['Признак клиента'] = hist_data_num_of_products['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента


fig_hist_num_of_products = px.histogram(
    data_frame=hist_data_num_of_products,
    x='NumOfProducts',
    color='Признак клиента',
    #histnorm='percent', #при необходимости можно убрать комментирование на строку для получения на оси доли в %, вместо количества клиентов
    title='Численность клиентов по количеству приобретенных услуг у банка'
)

fig_hist_num_of_products.update_xaxes(title_text='Количество приобретённых услуг', col=1, row=1) # добавляем подписи на оси 
fig_hist_num_of_products.update_yaxes(title_text='Количество клиентов', col=1, row=1)


fig_hist_num_of_products.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_hist_num_of_products.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig_hist_num_of_products.update_layout(bargap=0.2) #отступы между столбцами


fig_hist_num_of_products.show()

## Вывод по Вопросу №7. 
Большинство клиентов в обеих группах (ушедшие и лояльные клиенты) пользуются 1 или 2 видами услуг. При этом среди ушедших клиентов подавляющее большинство воспользовались 1 видом услуг. Лояльные клиенты в свою очередь чаще пользуются 2 видами услуг. Очевидно это связано с тем фактом, что если клиенту не понравится первая услуга, которой он воспользовался, то он с большей вероятностью уйдет и не будет пользоваться другими услугами. Лояльные клиенты, если их устраивает первая услуга, с большей долей вероятности будут пользоваться и другими услугами банка. 

## Вопрос №8. 
Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [291]:
hist_data_active = churn_data.loc[::, ['IsActiveMember', 'Exited']] #делаем новый df 

hist_data_active['Признак клиента'] = hist_data_active['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента
hist_data_active['Статус клиента'] = hist_data_active['IsActiveMember'].apply(lambda x: 'Активный клиент' if x==1 else 'Неактивный клиент') #преобразуем значения в столбце признак клиента

fig_hist_active = px.histogram(
    data_frame=hist_data_active,
    x='Статус клиента',
    color='Признак клиента',
    #histnorm='percent', #при необходимости можно убрать комментирование на строку для получения на оси доли в %, вместо количества клиентов
    title='Численность клиентов по статусу активности'
)

fig_hist_active.update_xaxes(title_text='Статус клиента', col=1, row=1) # добавляем подписи на оси 
fig_hist_active.update_yaxes(title_text='Количество клиентов', col=1, row=1)


fig_hist_active.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_hist_active.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig_hist_active.update_layout(bargap=0.2) #отступы между столбцами


fig_hist_active.show()

## Вывод по Вопросу №8. 
Ожидаемо, что если по статусу клиент активный, то он с меньшей вероятностью станет ушедшим клиентом. График только подтверждает это. Чтобы уменьшить отток клиентов среди неактивным можно предложить банку: а) пересмотреть методику отнесения клиентов к неактивным (в задании не указано по какому критерию они относятся к той или иной группе) таким образом, чтобы часть лояльных активных клиентов относилась к лояльным неактивным клиентам, либо часть ушедших неактивных относилась к ушедшим активным; б) провести повторную работу с ушедшими клиентами (как с активными, так и неактивными) и предложить им определённые условия по услугам (возможно это позволит перевести их в статус лояльных клиентов). Оба предложения позволят изменить структуру столбца "Неактивный клиент" и уменьшить в нем долю ушедших клиентов.


## Вопрос №9. 
В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.



In [292]:
heat_data = churn_data.pivot_table(
    values='Exited',
    index='Geography',
    fill_value=0
) # создаем df c долями клиентов

heat_data['Exited'] = round(heat_data['Exited'] * 100, 2) # переводим доли в % выражение

heat_data['Geography'] = heat_data.index
heat_data = heat_data.rename(columns={'Exited': 'Доля ушедших клиентов, %'}) # указываем соответствующие названия 


fig_heat =  px.choropleth(
    data_frame=heat_data, 
    locations='Geography', 
    locationmode = "country names", 
    color='Доля ушедших клиентов, %', 
    title='Картограмма стран по доле ушедших клиентов', 
    width=800, 
    height=500, 
    color_continuous_scale='Reds',
    scope='europe' #увидел, что все страны - только из Европы, поэтому решил показывать только карту Европы, вместо карты мира
)
fig_heat.show()

## Вывод по Вопросу №9. 
Наибольшая доля ушедших клиентов наблюдается в Германии - 32% от общего числа клиентов в этой стране. Вероятно это связано с тем, что в данной стране больше доля населения, которая имеет доходы, позволяющие выступать клиентами банковских организаций. Также это может быть связано с конкуренцией на данном географическом рынке: другие банки предлагают лучшие условия для более состоятельных клиентов. Не исключены и ментальные причины: "немецкая" бережливость и прагматичность заставляет клиентов из данного региона искать и выбирать лучшие предложения для своих финансовых целей.

## Вопрос №10. 
Переведите числовой признак CreditScore в категориальный. Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

Постройте сводную таблицу, строками которой являются категории кредитного рейтинга (CreditScoreCat), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (Tenure). В ячейках сводной таблицы должно находиться среднее по признаку оттока (Exited) — доля ушедших пользователей.

На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [293]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"
    
churn_data['CreditScoreCat'] =  churn_data['CreditScore'].apply(get_credit_score_cat)

churn_data['Exited'] = churn_data['Exited'] * 100 # приведем значение столбца в % выражение для дальнейшего использования в таблице

new_pivot = churn_data.pivot_table(
    values='Exited',
    index='CreditScoreCat',
    columns='Tenure',
    
    fill_value=0
) # cводная для тепловой карты


heatmap_creditcat = px.imshow(
    new_pivot,
    title='Teпловая карта ушедших клиентов, в % от соответствующей группы',
    color_continuous_scale='RdBu_r'  
)

heatmap_creditcat.update_xaxes(title_text='Сколько лет пользуется услугами', col=1, row=1) # добавляем подписи на оси 
heatmap_creditcat.update_yaxes(title_text='Кредитный рейтинг', col=1, row=1)


heatmap_creditcat.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
heatmap_creditcat.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

heatmap_creditcat.show()

## Вывод по Вопросу №10. 
Группы клиентов, в которых отмечается наибольшая доля ушедших клиентов - это клиенты, пользующиеся услугами до года и более 10 лет с очень плохим кредитным рейтингом (Very Poor). Доля ушедших клиентов в указанных группах - 44% и 33% соответственно. Вообще по всей группе клиентов с очень плохим кредитным рейтингом больше групп с повышенными долями ушедших клиентов (например, в группе клиентов со сроком 3 года - 28%, 4 года - 29%). Вероятно это связано с тем, что новым клиентам с плохим кредитным рейтингом банк предоставляет не самые выгодные услувия, стараясь защитить себя от рисков неплатежа. В дальнейшем учитывая динамику и результаты работы с клиентом банк пересматривает свои условия и доля ушедших клиентов снижается.